In [75]:
import spacy
!python -m spacy download en_core_web_lg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 400.7 MB 10 kB/s              
You should consider upgrading via the '/Users/aryopg/Library/Caches/pypoetry/virtualenvs/relation-extraction-paper-chat-TlqBbrrO-py3.9/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [76]:
nlp=spacy.load('en_core_web_lg')
nlp.pipe_names
# from spacy.language import Language
# nlp = spacy.load(
#     "/Users/aryopg/Documents/learning/relation-extraction-paper-chat/md_ner_classifier"
# )

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [77]:
import pickle
import kindred

corpus = kindred.load("pubannotation", "/Users/aryopg/Downloads/AGAC_training/json")

In [78]:
spacy_training_corpus = []
for doc in corpus.documents:
    text = doc.text
    entities = []
    for entity in doc.entities:
        entity_position = entity.position
        entity_type = entity.entityType
        entities += [(entity_position[0][0], entity_position[0][1], entity_type)]
    spacy_training_corpus += [(text, {"entities": list(set(entities))})]

In [79]:
pipe_exceptions = ["transformer", "ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
unaffected_pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [80]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

In [91]:
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        random.shuffle(spacy_training_corpus)
        losses = {}

        batches = minibatch(spacy_training_corpus, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                examples += [example]
            nlp.update(
                examples,
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print(f"Iteration: {iteration}; Losses: ", losses)

Iteration: 0; Losses:  {'ner': 3130.407035167657}
Iteration: 1; Losses:  {'ner': 3047.2624072784247}
Iteration: 2; Losses:  {'ner': 2960.83178543289}
Iteration: 3; Losses:  {'ner': 2976.3676830373456}
Iteration: 4; Losses:  {'ner': 2946.5863957378324}
Iteration: 5; Losses:  {'ner': 3011.5531580573825}
Iteration: 6; Losses:  {'ner': 2912.2483507757306}
Iteration: 7; Losses:  {'ner': 2860.424510903668}
Iteration: 8; Losses:  {'ner': 2807.3608337102332}
Iteration: 9; Losses:  {'ner': 2776.684142934905}
Iteration: 10; Losses:  {'ner': 2844.0118180559416}
Iteration: 11; Losses:  {'ner': 2859.5023181650367}
Iteration: 12; Losses:  {'ner': 2796.389600234484}
Iteration: 13; Losses:  {'ner': 2695.8637075458687}
Iteration: 14; Losses:  {'ner': 2644.7818246029638}
Iteration: 15; Losses:  {'ner': 2617.9777827933895}
Iteration: 16; Losses:  {'ner': 2669.9295745796976}
Iteration: 17; Losses:  {'ner': 2686.8644019632584}
Iteration: 18; Losses:  {'ner': 2610.8030630889534}
Iteration: 19; Losses:  {'ne

In [92]:
doc = nlp("Effects of a leukemia-associated gain-of-function mutation of SHP-2 phosphatase on interleukin-3 signaling.\nMutations in SHP-2 phosphatase that cause hyperactivation of its catalytic activity have been identified in human leukemias, particularly juvenile myelomonocytic leukemia, which is characterized by hypersensitivity of myeloid progenitor cells to granulocyte macrophage colony-stimulating factor and interleukin (IL)-3. However, the molecular mechanisms by which gain-of-function (GOF) mutations of SHP-2 induce hematopoietic malignancies are not fully understood. Our previous studies have shown that SHP-2 plays an essential role in IL-3 signal transduction in both catalytic-dependent and -independent manners and that overexpression (5-6-fold) of wild type (WT) SHP-2 attenuates IL-3-mediated hematopoietic cell function through accelerated dephosphorylation of STAT5. These results raised the possibility that SHP-2-associated leukemias are not solely attributed to the increased catalytic activity of GOF mutant SHP-2. GOF mutant SHP-2 must have gained additional capacities. To test this possibility, we investigated effects of a GOF mutation of SHP-2 (SHP-2 E76K) on hematopoietic cell function and IL-3 signal transduction by comparing with those of overexpressed WT SHP-2. Our results showed that SHP-2 E76K mutation caused myeloproliferative disease in mice, while overexpression of WT SHP-2 decreased hematopoietic potential of the transduced cells in recipient animals. The E76K mutation in the N-terminal Src homology 2 domain increased interactions of mutant SHP-2 with Grb2, Gab2, and p85, leading to hyperactivation of IL-3-induced Erk and phosphatidylinositol 3-kinase (PI3K) pathways. In addition, despite the substantial increase in the catalytic activity, dephosphorylation of STAT5 by SHP-2 E76K was dampened. Furthermore, catalytically inactive SHP-2 E76K with an additional C459S mutation retained the capability to increase the interaction with Gab2 and to enhance the activation of the PI3K pathway. Taken together, these studies suggest that in addition to the elevated catalytic activity, fundamental changes in physical and functional interactions between GOF mutant SHP-2 and signaling partners also play an important role in SHP-2-related leukemigenesis.")

In [93]:
for ent in doc.ents:
    print(f"{ent.text} - {ent.label_}")

SHP-2 - Gene
phosphatase - Enzyme
hyperactivation - PosReg
catalytic activity - MPA
leukemias - Disease
juvenile myelomonocytic leukemia - Disease
mutations - Var
SHP-2 - Gene
E76K - Var
SHP-2 - Gene
E76K mutation - Var
myeloproliferative disease - Disease
E76K mutation - Var
increased - PosReg
interactions - Interaction
mutant - Var
SHP-2 - Gene
Grb2 - Protein
hyperactivation - PosReg
IL-3-induced Erk and phosphatidylinositol 3-kinase (PI3K) pathways - Pathway
catalytic activity - MPA
dephosphorylation of STAT5 - MPA
SHP-2 - Gene
E76K - Var
dampened - NegReg
SHP-2 - Gene
E76K - Var
C459S - Var
increase - PosReg
interaction - Interaction
Gab2 - Protein
enhance - PosReg
activation - PosReg
PI3K pathway - Pathway
elevated - PosReg


In [94]:
doc = nlp("High-grade serous ovarian carcinoma (HGSOC) is clinically dominant and accounts for ~ 80% deaths in all types of ovarian cancer. The delayed diagnosis, rapid development, and wide dissemination of HGSOC collectively contribute to its high mortality rate and poor prognosis in the patients. Suppressors of cytokine signaling 7 (SOCS7) can regulate cytokine signaling and participate in cell cycle arrest and regulation of cell proliferation, which might also be involved in carcinogenesis. Here, we designated to investigate the functions and mechanisms of SOCS7 in HGSOC.")
for ent in doc.ents:
    print(f"{ent.text} - {ent.label_}")

High-grade serous ovarian carcinoma - Disease
HGSOC - Gene


In [95]:
nlp.to_disk("lg_ner_classifier_90")

In [16]:
# import kindred

In [18]:
# entities = []
# for entity in doc.ents:
#     entities += [kindred.Entity(entity.label_, entity.text, [(entity.start_char, entity.end_char)])]
# kindred_doc = kindred.Document(text=doc.text, entities=entities)

In [19]:
# kindred_doc

<Document Effects of a leukemia-associated gain-of-function mutation of SHP-2 phosphatase on interleukin-3 signaling.
Mutations in SHP-2 phosphatase that cause hyperactivation of its catalytic activity have been identified in human leukemias, particularly juvenile myelomonocytic leukemia, which is characterized by hypersensitivity of myeloid progenitor cells to granulocyte macrophage colony-stimulating factor and interleukin (IL)-3. However, the molecular mechanisms by which gain-of-function (GOF) mutations of SHP-2 induce hematopoietic malignancies are not fully understood. Our previous studies have shown that SHP-2 plays an essential role in IL-3 signal transduction in both catalytic-dependent and -independent manners and that overexpression (5-6-fold) of wild type (WT) SHP-2 attenuates IL-3-mediated hematopoietic cell function through accelerated dephosphorylation of STAT5. These results raised the possibility that SHP-2-associated leukemias are not solely attributed to the increase

In [22]:
# kindred_corpus = kindred.Corpus()
# kindred_corpus.addDocument(kindred_doc)

In [55]:
# with open("classifier.pickle", "rb") as model_file:
#     re_model = pickle.load(model_file)

In [56]:
# re_model.predict(kindred_corpus)

In [57]:
# kindred_corpus.getRelations()

[<Relation CauseOf [<Entity Var:'Mutations' sourceid=None [(108, 117)]>, <Entity PosReg:'hyperactivation' sourceid=None [(150, 165)]>] ['subj', 'obj']>,
 <Relation ThemeOf [<Entity Gene:'SHP-2' sourceid=None [(121, 126)]>, <Entity Var:'Mutations' sourceid=None [(108, 117)]>] ['subj', 'obj']>,
 <Relation ThemeOf [<Entity Enzyme:'phosphatase' sourceid=None [(127, 138)]>, <Entity Var:'Mutations' sourceid=None [(108, 117)]>] ['subj', 'obj']>,
 <Relation ThemeOf [<Entity MPA:'catalytic activity' sourceid=None [(173, 191)]>, <Entity PosReg:'hyperactivation' sourceid=None [(150, 165)]>] ['subj', 'obj']>,
 <Relation ThemeOf [<Entity Disease:'leukemias' sourceid=None [(222, 231)]>, <Entity MPA:'catalytic activity' sourceid=None [(173, 191)]>] ['subj', 'obj']>,
 <Relation ThemeOf [<Entity Disease:'juvenile myelomonocytic leukemia' sourceid=None [(246, 278)]>, <Entity MPA:'catalytic activity' sourceid=None [(173, 191)]>] ['subj', 'obj']>,
 <Relation ThemeOf [<Entity Gene:'SHP-2' sourceid=None [(1

In [29]:
nlp.to_disk("trf_ner_classifier")

In [30]:
# doc2 = nlp("There has been much progress in ovarian cancer screening and prevention in recent years. Improved tools that combine genetic and epidemiologic factors to predict an individual's ovarian cancer risk are set to become available for tailoring preventive and screening approaches. The increasing evidence on tubal origins of a proportion of ovarian cancer has paved the way to use of opportunistic bilateral salpingectomy at tubal ligation and hysterectomy in the general population. Clinical trials are in progress to estimate the long-term effects on endocrine function. In women at high risk, risk reducing salpingo-oophorectomy remains the standard of care with the current focus on management of resulting noncancer outcomes, especially sexual dysfunction in younger women. This has led to evaluation of early bilateral salpingectomy and delayed oophorectomy in this population. Meanwhile, modeling suggests that BRCA mutation carriers should consider using the oral contraceptive pill for chemoprevention. In the general population, the largest ovarian cancer screening trial to date, the UK Collaborative Trial of Ovarian Cancer Screening reported a stage shift with annual multimodal screening using the longitudinal CA 125 Risk of Ovarian Cancer Algorithm but not with annual transvaginal ultrasound screening. There was no definitive mortality reduction with either screening strategy compared with no screening. Further follow-up until December 2018 in now underway. Stage shift and higher rates of optimal cytoreduction were also reported during 3- to 4-monthly multimodal screening in the United Kingdom and U.S. high-risk screening trials. Although all agree that there is not yet evidence to support general population screening, recommendations for high-risk screening vary between countries. A key finding from the screening trials has been the better performance of longitudinal algorithms compared with a single cutoff for CA 125. A major focus of ovarian cancer biomarker discovery work has been tumor DNA markers in both plasma and novel specimens such as cervical cytology samples.")

In [34]:
# doc2

There has been much progress in ovarian cancer screening and prevention in recent years. Improved tools that combine genetic and epidemiologic factors to predict an individual's ovarian cancer risk are set to become available for tailoring preventive and screening approaches. The increasing evidence on tubal origins of a proportion of ovarian cancer has paved the way to use of opportunistic bilateral salpingectomy at tubal ligation and hysterectomy in the general population. Clinical trials are in progress to estimate the long-term effects on endocrine function. In women at high risk, risk reducing salpingo-oophorectomy remains the standard of care with the current focus on management of resulting noncancer outcomes, especially sexual dysfunction in younger women. This has led to evaluation of early bilateral salpingectomy and delayed oophorectomy in this population. Meanwhile, modeling suggests that BRCA mutation carriers should consider using the oral contraceptive pill for chemoprev